# Run Polina's code for 100 years of jazz

## Create an RDF repository for JE

In [ ]:
# Input: DATA/JECompleteIndex_cleaned.csv
# Output: TTL/JE_PyRDF.ttl, TTL/JE_PyRDF_tmp.ttl
%run JEtoRDF_V6

## Add leaders to the JE repository

In [ ]:
# Input: TTL/JE_PyRDF.ttl (output of previous step only)
# Output: TTL/JE_leaders.ttl
exec(open('match_leaders.py').read(), {'PROPERTY_PREFIX': 'je'})

## Add fingerprints for JE

In [ ]:
# Input: DATA/id_dtl1000_idonly.csv, TTL/JE_leaders.ttl
# Output: TTL/JE_fprints.ttl
%run JE_add_short_fprints

## Create an RDF repository from the json files for the (subset of) Illinois dataset
This is the slowest part, taking ~90 minutes on a good laptop.

In [ ]:
# Input: DATA/DTL1000_1960-2020_json_v0/???0s.csv_110_musinstr.json
# Output: TTL/ILL1000.ttl
%run Ill1000_2RDF

## Add bands to the ILL repository from the SQL (Lord Discography) database 
Download the SQL database from OSF

In [ ]:
# Input: DATA/2019.10.24_disco.sqlite, DATA/DTL1000_1960-2020_json_v0/???0s.csv_110_musinstr.json, TTL/ILL1000.ttl
# Output: TTL/ILL_bands.ttl
%run Ill1000_add_bands_from_sql

## Add leaders to the ILL repository

In [ ]:
# Input: TTL/ILL_bands.ttl
# Output: TTL/ILL_leaders.ttl
exec(open('match_leaders.py').read(), {'PROPERTY_PREFIX': 'lord'})

## For both JE and ILL repositories (separately) add DTL instrument labels

In [ ]:
# Input: DATA/orig2DTL_instruments.csv, TTL/JE_fprints.ttl
# Ouput: TTL/JE_inst.ttl
exec(open('map_instruments.py').read(), {'PROPERTY_PREFIX': 'je'})

# Input: DATA/orig2DTL_instruments.csv, TTL/ILL_leaders.ttl
# Ouput: TTL/ILL_inst.ttl
exec(open('map_instruments.py').read(), {'PROPERTY_PREFIX': 'lord'})  # 'je', 'lord', 'orig'

## Add solo information for JE

In [ ]:
# Input: DATA/solo_extract_meta.csv, DATA/DTLtoJE_instruments.csv, TTL/JE_inst.ttl
# Output: TTL/JE_solos.ttl, PyLOG/JE_no_musicians_found.json, PyLOG/JE_no_matching_instrument_found.json, PyLOG/JE_multiple_musicians_found.json
%run JE1000_add_solos

## Add solo information for ILL

In [ ]:
# Input: DATA/solo_extract_meta.csv, TTL/ILL_inst.ttl
# Output: TTL/ILL_solos.ttl, PyLOG/Ill_no_musicians_found.json, PyLOG/Ill_no_matching_instrument_found.json, PyLOG/Ill_multiple_musicians_found.json
%run Ill1000_add_solos

## Add styles

In [ ]:
# Input: DATA/styles.csv, TTL/JE_solos.ttl
# Output: TTL/JE_styles.ttl
exec(open('add_styles.py').read(), {'PROPERTY_PREFIX': 'je'})

# Input: DATA/styles.csv, TTL/ILL_solos.ttl
# Output: TTL/ILL_styles.ttl
exec(open('add_styles.py').read(), {'PROPERTY_PREFIX': 'lord'})

## Add links to LOD

In [ ]:
# Input: DATA/LJpeople.nt, TTL/JE_styles.ttl
# Output: TTL/JE_LODpeople.ttl
exec(open('DTLtoLJpeople.py').read(), {'PROPERTY_PREFIX': 'je'})

# Input: DATA/LJpeople.nt, TTL/ILL_styles.ttl
# Output: TTL/ILL_LODpeople.ttl
exec(open('DTLtoLJpeople.py').read(), {'PROPERTY_PREFIX': 'lord'})

## Merge JE and ILL repositories
Musicians, Bands, Tunes and Instruments are merged by name/title.
Sessions and performances are not merged since they do not overlap between JE and ILL.
Merging is not necessary for the interface to work, just combining the two files would suffice.

In [ ]:
# Input: TTL/JE_LODpeople.ttl, TTL/ILL_LODpeople.ttl
# Output: TTL/JE_ILL_merged.ttl
%run merging

## Patch metadata to resolve missing solo performers

In [ ]:
# Input: DATA/missing_performer.csv, TTL/JE_ILL_merged.ttl
# Output: TTL/JE_ILL_patched.ttl
%run patch_metadata

## Extract the relevant triples from the merged "100 years of jazz" turtle file

In [ ]:
# Input: DATA/id_dtl1000_idonly.csv, TTL/JE_ILL_patched.ttl
# Output: TTL/dtl1000.ttl
%run dtl1000.py